In [15]:
import datetime
import sys
import pandas as pd
import sqlalchemy
import numpy
import gc
import json
import pprint
from poseidon import poseidon
from datetime import datetime , timedelta
from dateutil.relativedelta import relativedelta

#import commond.ipynb from same folder
import import_ipynb
from conf_ppe import *
from common import *

In [16]:
def getMeasurePoint_21(deviceId, startTime, endTime, interval, measurement_point):
    '''This function calls get the data as per asset and time duration provided. Interval is a mandatory filed. '''
    try:
        urlData = "%s/tsdb-service/v2.1/ai-normalized?orgId=%s" % (address, orgId)
        requestBody = {
            "assetIds": "%s" % deviceId,
            "measurepointsWithLogic": "avg(" + measurement_point + ")",
            "startTime": "%s" % startTime,
            "endTime": "%s" % endTime,
            "interval": "%s" % interval,
            "localTimeAccuracy": "true",
            "localTimeFormat": 0,
            "itemFormat": 2
        }
        req = poseidon.urlopen(appkey, appsecret, urlData, requestBody)
        print(urlData, requestBody )
        if req is None: 
            print(urlData, requestBody )
            return None
        df = pd.DataFrame()
        if req['code'] == 80400:
            value = 'NA'
        elif req['data']['items']:
            if req is not None:
                for item in req['data']['items']:
                    #print(item)
                    value = item['value']
                    d = {'localtime': item['localtime'], measurement_point: item['value']}
                    df = pd.DataFrame(d)
                    return df
            else:
                value = 0
        else:
            value = 0
        return df
    except Exception as err:
        print(err)

In [17]:
def getAssetNodes(treeId):
    try:
        urlAsset = '%s/asset-tree-service/v2.1/asset-nodes?action=searchRelatedAsset&treeId=%s&orgId=%s' % (
            address, treeId, orgId)
        requestBodyAsset = {
            "action": "searchRelatedAsset",
            "projection": ["assetId", "name"]
        }
        req = poseidon.urlopen(appkey, appsecret, urlAsset, requestBodyAsset)
        return req;
    except Exception as err:
        print(err)    

In [18]:
def getMeasurePoint_20(deviceId, startTime, endTime, interval, measurement_point):
    '''This function calls get the data as per asset and time duration provided. Interval is a mandatory filed. '''
    try:
        urlData = "%s/tsdb-service/v2.0/ai-normalized?orgId=%s&assetIds=%s&measurepointsWithLogic=avg(%s)&interval=%s&startTime=%s&endTime=%s&pageSize=2000&localTimeAccuracy=%s" \
        % (address, orgId, deviceId, measurement_point, interval,startTime, endTime, "true")
        req = poseidon.urlopen(appkey, appsecret, urlData)
        
        if req is None: 
            print(urlData )
            return None
        #print(req)
        df = pd.DataFrame()
        if req['msg'] != 'success':
            value = 'NA'
        elif req['data']['items']:
            df = pd.DataFrame(req['data']['items'])
        else:
            value = 0
        return df
    except Exception as err:
        print(err)

In [19]:
def getMeasurePoint_rawdata_20(deviceId, apitype, startTime, endTime, interval, measurement_point):
    '''This function calls get the data as per asset and time duration provided. Interval is a mandatory filed. '''
    try:
        urlData = "%s/tsdb-service/v2.0/%s?orgId=%s&assetIds=%s&measurepoints=%s&interval=%s&startTime=%s&endTime=%s&pageSize=60000&localTimeAccuracy=%s" \
        % (address, apitype, orgId, deviceId, measurement_point, interval,startTime, endTime, "true")
        req = poseidon.urlopen(appkey, appsecret, urlData)
        
        if req is None: 
            print(urlData )
            return None
        #print(req)
        df = pd.DataFrame()
        if req['msg'] != 'success':
            value = 'NA'
        elif req['data']['items']:
            df = pd.DataFrame(req['data']['items'])
        else:
            value = 0
        return df
    except Exception as err:
        print(err)

In [6]:
#for Active_Energy, must use generic API
#for Active_Power, must use ai API
res = {}
req = getAssetNodes(treeId)
startTime = "2020-11-11%2000:00:00"
endTime = "2020-11-11%2023:59:59"
mpName = "Active_Energy"
for val in req['data']:
    
    assetId = val['assetId']
    assetName = val['name']['defaultValue']
    
    if assetId in substationIds:
        print(val)
        ds = getMeasurePoint_rawdata_20(assetId, "generic", startTime, endTime, 1, mpName)
        res.update({assetName: ds})
        print(assetId, res)

{'assetId': 'ooBROdU2', 'name': {'i18nValue': {}, 'defaultValue': 'MainSub_03'}}
ooBROdU2 {'MainSub_03': Empty DataFrame
Columns: []
Index: []}
{'assetId': 'rgzLjVwc', 'name': {'i18nValue': {}, 'defaultValue': 'MainSub_02'}}
rgzLjVwc {'MainSub_03': Empty DataFrame
Columns: []
Index: [], 'MainSub_02': Empty DataFrame
Columns: []
Index: []}
{'assetId': 'ZcOrVdNa', 'name': {'i18nValue': {}, 'defaultValue': 'MainSub_01'}}
HTTP Error 504: Gateway Time-out
https://apim-ppe1.envisioniot.com/tsdb-service/v2.0/generic?orgId=o15897810838641&assetIds=ZcOrVdNa&measurepoints=Active_Energy&interval=1&startTime=2020-11-11%2000:00:00&endTime=2020-11-11%2023:59:59&pageSize=60000&localTimeAccuracy=true
ZcOrVdNa {'MainSub_03': Empty DataFrame
Columns: []
Index: [], 'MainSub_02': Empty DataFrame
Columns: []
Index: [], 'MainSub_01': None}


In [20]:

def export_measurement_point(assetName, apitype, assetId, beginDate, endDate, mpName, folder_name):    
    ds = pd.DataFrame()    
    date = datetime.strptime(beginDate, '%Y-%m-%d %H:%M:%S')
    delta = datetime.strptime(endDate, '%Y-%m-%d %H:%M:%S') - date
    for i in range(delta.days): 
        dayBefore = date
        print(dayBefore)
        date += relativedelta(days=1)
        print(date)
        if (date > datetime.strptime(endDate, '%Y-%m-%d %H:%M:%S')) == True:
            break

        val = getMeasurePoint_rawdata_20(assetId, apitype, str(dayBefore).replace(" ", "%20"), str(date).replace(" ", "%20"), 1, mpName)
        ds = ds.append(val)

    print(ds.head(5))
    print(ds.shape)

    # export the mp to files by readiable file name
    export_dataset(ds,assetName + '-' + assetId + '-' + mpName, folder_name)
    del ds
    gc.collect()

In [26]:
beginDate = "2019-10-10 00:00:01"
endDate = "2020-11-13 17:00:00"
mpName = "Active_Energy"
apiName = "generic"
req = getAssetNodes(treeId)

for val in req['data']:
    assetId = val['assetId']
    assetName = val['name']['defaultValue']
    
    if assetId in substationIds: 
        print("Download Mp of {} {}".format( assetName, assetId ))
        export_measurement_point(assetName, apiName , assetId, beginDate, endDate, mpName, folder_name)

Download Mp of MainSub_03 2Z86rfOS
2019-10-10 00:00:01
2019-10-11 00:00:01
2019-10-11 00:00:01
2019-10-12 00:00:01
2019-10-12 00:00:01
2019-10-13 00:00:01
2019-10-13 00:00:01
2019-10-14 00:00:01
2019-10-14 00:00:01
2019-10-15 00:00:01
2019-10-15 00:00:01
2019-10-16 00:00:01
2019-10-16 00:00:01
2019-10-17 00:00:01
2019-10-17 00:00:01
2019-10-18 00:00:01
2019-10-18 00:00:01
2019-10-19 00:00:01
2019-10-19 00:00:01
2019-10-20 00:00:01
2019-10-20 00:00:01
2019-10-21 00:00:01
2019-10-21 00:00:01
2019-10-22 00:00:01
2019-10-22 00:00:01
2019-10-23 00:00:01
2019-10-23 00:00:01
2019-10-24 00:00:01
2019-10-24 00:00:01
2019-10-25 00:00:01
2019-10-25 00:00:01
2019-10-26 00:00:01
2019-10-26 00:00:01
2019-10-27 00:00:01
2019-10-27 00:00:01
2019-10-28 00:00:01
2019-10-28 00:00:01
2019-10-29 00:00:01
2019-10-29 00:00:01
2019-10-30 00:00:01
2019-10-30 00:00:01
2019-10-31 00:00:01
2019-10-31 00:00:01
2019-11-01 00:00:01
2019-11-01 00:00:01
2019-11-02 00:00:01
2019-11-02 00:00:01
2019-11-03 00:00:01
2019-

2020-05-01 00:00:01
2020-05-02 00:00:01
2020-05-02 00:00:01
2020-05-03 00:00:01
2020-05-03 00:00:01
2020-05-04 00:00:01
2020-05-04 00:00:01
2020-05-05 00:00:01
2020-05-05 00:00:01
2020-05-06 00:00:01
2020-05-06 00:00:01
2020-05-07 00:00:01
2020-05-07 00:00:01
2020-05-08 00:00:01
2020-05-08 00:00:01
2020-05-09 00:00:01
2020-05-09 00:00:01
2020-05-10 00:00:01
2020-05-10 00:00:01
2020-05-11 00:00:01
2020-05-11 00:00:01
2020-05-12 00:00:01
2020-05-12 00:00:01
2020-05-13 00:00:01
2020-05-13 00:00:01
2020-05-14 00:00:01
2020-05-14 00:00:01
2020-05-15 00:00:01
2020-05-15 00:00:01
2020-05-16 00:00:01
2020-05-16 00:00:01
2020-05-17 00:00:01
2020-05-17 00:00:01
2020-05-18 00:00:01
2020-05-18 00:00:01
2020-05-19 00:00:01
2020-05-19 00:00:01
2020-05-20 00:00:01
2020-05-20 00:00:01
2020-05-21 00:00:01
2020-05-21 00:00:01
2020-05-22 00:00:01
2020-05-22 00:00:01
2020-05-23 00:00:01
2020-05-23 00:00:01
2020-05-24 00:00:01
2020-05-24 00:00:01
2020-05-25 00:00:01
2020-05-25 00:00:01
2020-05-26 00:00:01


Download Mp of MainSub_02 5QM4Useo
2019-10-10 00:00:01
2019-10-11 00:00:01
2019-10-11 00:00:01
2019-10-12 00:00:01
2019-10-12 00:00:01
2019-10-13 00:00:01
2019-10-13 00:00:01
2019-10-14 00:00:01
2019-10-14 00:00:01
2019-10-15 00:00:01
2019-10-15 00:00:01
2019-10-16 00:00:01
2019-10-16 00:00:01
2019-10-17 00:00:01
2019-10-17 00:00:01
2019-10-18 00:00:01
2019-10-18 00:00:01
2019-10-19 00:00:01
2019-10-19 00:00:01
2019-10-20 00:00:01
2019-10-20 00:00:01
2019-10-21 00:00:01
2019-10-21 00:00:01
2019-10-22 00:00:01
2019-10-22 00:00:01
2019-10-23 00:00:01
2019-10-23 00:00:01
2019-10-24 00:00:01
2019-10-24 00:00:01
2019-10-25 00:00:01
2019-10-25 00:00:01
2019-10-26 00:00:01
2019-10-26 00:00:01
2019-10-27 00:00:01
2019-10-27 00:00:01
2019-10-28 00:00:01
2019-10-28 00:00:01
2019-10-29 00:00:01
2019-10-29 00:00:01
2019-10-30 00:00:01
2019-10-30 00:00:01
2019-10-31 00:00:01
2019-10-31 00:00:01
2019-11-01 00:00:01
2019-11-01 00:00:01
2019-11-02 00:00:01
2019-11-02 00:00:01
2019-11-03 00:00:01
2019-

2020-05-01 00:00:01
2020-05-02 00:00:01
2020-05-02 00:00:01
2020-05-03 00:00:01
2020-05-03 00:00:01
2020-05-04 00:00:01
2020-05-04 00:00:01
2020-05-05 00:00:01
2020-05-05 00:00:01
2020-05-06 00:00:01
2020-05-06 00:00:01
2020-05-07 00:00:01
2020-05-07 00:00:01
2020-05-08 00:00:01
2020-05-08 00:00:01
2020-05-09 00:00:01
2020-05-09 00:00:01
2020-05-10 00:00:01
2020-05-10 00:00:01
2020-05-11 00:00:01
2020-05-11 00:00:01
2020-05-12 00:00:01
2020-05-12 00:00:01
2020-05-13 00:00:01
2020-05-13 00:00:01
2020-05-14 00:00:01
2020-05-14 00:00:01
2020-05-15 00:00:01
2020-05-15 00:00:01
2020-05-16 00:00:01
2020-05-16 00:00:01
2020-05-17 00:00:01
2020-05-17 00:00:01
2020-05-18 00:00:01
2020-05-18 00:00:01
2020-05-19 00:00:01
2020-05-19 00:00:01
2020-05-20 00:00:01
2020-05-20 00:00:01
2020-05-21 00:00:01
2020-05-21 00:00:01
2020-05-22 00:00:01
2020-05-22 00:00:01
2020-05-23 00:00:01
2020-05-23 00:00:01
2020-05-24 00:00:01
2020-05-24 00:00:01
2020-05-25 00:00:01
2020-05-25 00:00:01
2020-05-26 00:00:01


Download Mp of MainSub_01 bu4eboNR
2019-10-10 00:00:01
2019-10-11 00:00:01
2019-10-11 00:00:01
2019-10-12 00:00:01
2019-10-12 00:00:01
2019-10-13 00:00:01
2019-10-13 00:00:01
2019-10-14 00:00:01
2019-10-14 00:00:01
2019-10-15 00:00:01
2019-10-15 00:00:01
2019-10-16 00:00:01
2019-10-16 00:00:01
2019-10-17 00:00:01
2019-10-17 00:00:01
2019-10-18 00:00:01
2019-10-18 00:00:01
2019-10-19 00:00:01
2019-10-19 00:00:01
2019-10-20 00:00:01
2019-10-20 00:00:01
2019-10-21 00:00:01
2019-10-21 00:00:01
2019-10-22 00:00:01
2019-10-22 00:00:01
2019-10-23 00:00:01
2019-10-23 00:00:01
2019-10-24 00:00:01
2019-10-24 00:00:01
2019-10-25 00:00:01
2019-10-25 00:00:01
2019-10-26 00:00:01
2019-10-26 00:00:01
2019-10-27 00:00:01
2019-10-27 00:00:01
2019-10-28 00:00:01
2019-10-28 00:00:01
2019-10-29 00:00:01
2019-10-29 00:00:01
2019-10-30 00:00:01
2019-10-30 00:00:01
2019-10-31 00:00:01
2019-10-31 00:00:01
2019-11-01 00:00:01
2019-11-01 00:00:01
2019-11-02 00:00:01
2019-11-02 00:00:01
2019-11-03 00:00:01
2019-

2020-05-01 00:00:01
2020-05-02 00:00:01
2020-05-02 00:00:01
2020-05-03 00:00:01
2020-05-03 00:00:01
2020-05-04 00:00:01
2020-05-04 00:00:01
2020-05-05 00:00:01
2020-05-05 00:00:01
2020-05-06 00:00:01
2020-05-06 00:00:01
2020-05-07 00:00:01
2020-05-07 00:00:01
2020-05-08 00:00:01
2020-05-08 00:00:01
2020-05-09 00:00:01
2020-05-09 00:00:01
2020-05-10 00:00:01
2020-05-10 00:00:01
2020-05-11 00:00:01
2020-05-11 00:00:01
2020-05-12 00:00:01
2020-05-12 00:00:01
2020-05-13 00:00:01
2020-05-13 00:00:01
2020-05-14 00:00:01
2020-05-14 00:00:01
2020-05-15 00:00:01
2020-05-15 00:00:01
2020-05-16 00:00:01
2020-05-16 00:00:01
2020-05-17 00:00:01
2020-05-17 00:00:01
2020-05-18 00:00:01
2020-05-18 00:00:01
2020-05-19 00:00:01
2020-05-19 00:00:01
2020-05-20 00:00:01
2020-05-20 00:00:01
2020-05-21 00:00:01
2020-05-21 00:00:01
2020-05-22 00:00:01
2020-05-22 00:00:01
2020-05-23 00:00:01
2020-05-23 00:00:01
2020-05-24 00:00:01
2020-05-24 00:00:01
2020-05-25 00:00:01
2020-05-25 00:00:01
2020-05-26 00:00:01


In [27]:
parsed = getAssetNodes(treeId)
print(json.dumps(parsed, indent=2, sort_keys=True))

{
  "code": 0,
  "data": [
    {
      "assetId": "HlFebW8t",
      "name": {
        "defaultValue": "Main_substation",
        "i18nValue": {}
      }
    },
    {
      "assetId": "VHXwoLc4",
      "name": {
        "defaultValue": "Pioneer Zone",
        "i18nValue": {}
      }
    },
    {
      "assetId": "ZcOrVdNa",
      "name": {
        "defaultValue": "MainSub_01",
        "i18nValue": {}
      }
    },
    {
      "assetId": "ooBROdU2",
      "name": {
        "defaultValue": "MainSub_03",
        "i18nValue": {}
      }
    },
    {
      "assetId": "rgzLjVwc",
      "name": {
        "defaultValue": "MainSub_02",
        "i18nValue": {}
      }
    }
  ],
  "msg": "OK",
  "pagination": {
    "pageNo": 1,
    "pageSize": 100,
    "sortedBy": null,
    "totalSize": 5
  },
  "requestId": "cbefcbbe-bae1-4ba6-9f9c-24fe5980a70a"
}


In [69]:
dataA = {key: res[key] for key in substation if key in res}

In [70]:
print(dataA)

{'MainSub_01': None, 'MainSub_03': None, 'MainSub_02': None}


In [16]:
from datetime import datetime , timedelta 
date = datetime.strptime("2020-11-09 17:00:00", '%Y-%m-%d %H:%M:%S')
for i in range(5): 
    date += timedelta(days=1)
    print(date) 

2020-11-10 17:00:00
2020-11-11 17:00:00
2020-11-12 17:00:00
2020-11-13 17:00:00
2020-11-14 17:00:00


In [14]:
sgms_config = '1PyW1O8d'
parsed = (getAssetNodes(sgms_config))

print(json.dumps(parsed, indent=2, sort_keys=True))

{
  "code": 0,
  "data": [
    {
      "assetId": "NaBCekNm",
      "name": {
        "defaultValue": "vpp_config",
        "i18nValue": {}
      }
    },
    {
      "assetId": "OFQ4jR0q",
      "name": {
        "defaultValue": "doe_config_1",
        "i18nValue": {}
      }
    },
    {
      "assetId": "tw79ULYL",
      "name": {
        "defaultValue": "doe_config_2",
        "i18nValue": {}
      }
    },
    {
      "assetId": "uok2uxtN",
      "name": {
        "defaultValue": "Global",
        "i18nValue": {}
      }
    },
    {
      "assetId": "zUn1QecP",
      "name": {
        "defaultValue": "SGMS Configuration",
        "i18nValue": {}
      }
    }
  ],
  "msg": "OK",
  "pagination": {
    "pageNo": 1,
    "pageSize": 100,
    "sortedBy": null,
    "totalSize": 5
  },
  "requestId": "28efa75a-914d-4202-afab-d46ca063ad1b"
}


In [8]:
def getAssetDevice(assetId):
    try:
        urlAsset = '%s/asset-service/v2.1/assets?action=get&assetId=%s&orgId=%s' % (
            address, assetId, orgId)
        req = poseidon.urlopen(appkey, appsecret, urlAsset)
        return req;
    except Exception as err:
        print(err)    

In [9]:
parsed = getAssetDevice('tw79ULYL')
print(json.dumps(parsed, indent=2, sort_keys=True))

{
  "code": 0,
  "data": {
    "assetId": "tw79ULYL",
    "attributes": {
      "agc_factor": 0.0,
      "cap_contract": 6.0,
      "cap_rating": 25.5,
      "dch_cost": 0.0,
      "eff_c": 0.0,
      "eff_d": 0.0,
      "energy_cap": 0.0,
      "energy_price": 214.3,
      "grid_connection_list_Name": "PPT INTAKE 3 SUBSTATION NO. 1",
      "grid_name": "Singapore Power",
      "location": "PPT INTAKE 3 SUBSTATION",
      "loss_coeff": 1.0,
      "max_num_switch": 10000.0,
      "oper_mode": "COOPT",
      "p_max": 0.0,
      "p_min": 0.0,
      "r_max": 1.0,
      "r_min": 0.0,
      "soc_end_max": 90.0,
      "soc_end_min": 10.0,
      "soc_max": 90.0,
      "soc_min": 10.0,
      "switch_cost": 0.0,
      "time_interval": "15min",
      "time_limit": "120s",
      "time_window": "24h",
      "timezone": "UTC+8",
      "unctr_cost": 13.35,
      "zone_list": "PPT INTAKE 3 SUBSTATION"
    },
    "description": "",
    "modelId": "DOE_config",
    "modelIdPath": "/DOE_config",
    "nam

In [21]:
def getModelDefail(modelId):
    try:
        urlAsset = '%s/model-service/v2.1/thing-models?action=get&modelId=%s&orgId=%s' % (
            address, modelId, orgId)
        req = poseidon.urlopen(appkey, appsecret, urlAsset)
        return req;
    except Exception as err:
        print(err)  

In [22]:
parsed = getModelDefail('DOE_config')

print(json.dumps(parsed, indent=2, sort_keys=True))

{
  "code": 0,
  "data": {
    "attributes": {
      "agc_factor": {
        "dataDefinition": null,
        "dataType": "DOUBLE",
        "defaultValue": 0.0,
        "desc": "Coefficient to project the impact on the SOC following the AGC signal, the range of the value should be [-1,1]",
        "identifier": "agc_factor",
        "isRequired": false,
        "isStdElement": false,
        "name": {
          "defaultValue": "agc_factor",
          "i18nValue": {}
        },
        "stdElementId": null,
        "tags": {
          "group": "bess",
          "timeseries": "true",
          "visible": "true"
        },
        "unit": null
      },
      "cap_contract": {
        "dataDefinition": null,
        "dataType": "DOUBLE",
        "defaultValue": 6.0,
        "desc": "Contracted capacity. Terminal 4: 6MW; \nTerminal 5&6: 22MW. 22mW Contracted capacity lasts until Sep 18 2022. 6mW Contracted capacity lasts until Dec 02 2025.",
        "identifier": "cap_contract",
        "isR

In [11]:
parsed = getAssetDevice('O77Zvqmy')
print(json.dumps(parsed, indent=2, sort_keys=True))

{
  "code": 0,
  "data": {
    "assetId": "O77Zvqmy",
    "attributes": {},
    "description": "",
    "modelId": "PSA_SGMS_Terminal",
    "modelIdPath": "/PSA_SGMS_Terminal",
    "name": {
      "defaultValue": "PPT4",
      "i18nValue": {}
    },
    "orgId": "o15897810838641",
    "tags": {},
    "timezone": "+08:00"
  },
  "msg": "OK",
  "requestId": "3880e909-efe3-4784-8202-3a2fb0bb95ec"
}
